In [11]:
from haystack.nodes.base import BaseComponent
import requests

In [24]:
class YelpAPINode(BaseComponent):
    outgoing_edges = 1

    def __init__(self, api_key):
        self.api_key = api_key

    def run(self, query: str):
        # Use the query (or tokens) to make an API call to Yelp (or any other API)
        headers = {
            "Authorization": f"Bearer {self.api_key}"
        }
        
        # Assuming you want to search for businesses on Yelp based on the query
        response = requests.get(f"https://api.yelp.com/v3/businesses/search?location=Changi%20City%20Point&term=indian%20food&sort_by=best_match&limit=20", headers=headers)
        
        # Convert the response to JSON
        result_json = response.json()

        return {"query": query, "results": result_json}, "yelp_output1"
    
    def run_batch(self, queries: [str]):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [25]:
from haystack.pipelines import Pipeline

# Initialize the custom Yelp node
yelp_node = YelpAPINode(api_key="DmruoNQKEAtr4rNSjmhkN9z3SLoJTH9nfG2pGMVcCRY0nNBs1_Mqm0mk0O37e6YwqzQUCa0ayU3mMEA-yR003NkWQslarfo7dVjIDbMl77lPyXgf1zifhZ_4T2odZXYx")

In [26]:
# Create a Haystack pipeline
p = Pipeline()
p.add_node(component=yelp_node, name="YelpNode", inputs=["Query"])

In [27]:
result = p.run(query="pizza near me")
print(result)

{'query': 'pizza near me', 'results': {'businesses': [{'id': 'enbavlLlsJPiW8qvfp0nyQ', 'alias': 'mirchi-singapore-2', 'name': 'Mirchi', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/uYsG81k3lAJMbg0a3uYzMQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/mirchi-singapore-2?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg', 'review_count': 1, 'categories': [{'alias': 'indpak', 'title': 'Indian'}], 'rating': 4.0, 'coordinates': {'latitude': 1.35282, 'longitude': 103.985939}, 'transactions': [], 'location': {'address1': '65 Airport Boulevard', 'address2': '', 'address3': '', 'city': 'Singapore', 'zip_code': '819663', 'country': 'SG', 'state': 'SG', 'display_address': ['65 Airport Boulevard', 'Singapore 819663', 'Singapore']}, 'phone': '', 'display_phone': '', 'distance': 3638.7255167511303}, {'id': 'SEsG2lb0UkE36Z7LeHX9AA', 'alias': 'sankranti-singapore-2', 'name': 'Sankranti', 'image_url

In [23]:
import urllib.parse

def construct_yelp_url(params: dict) -> str:
    base_url = "https://api.yelp.com/v3/businesses/search?"
    query_parameters = []

    for key, value in params.items():
        # If the value is a list (e.g., for attributes or price levels), 
        # we need to add each value separately with the same key.
        if isinstance(value, list):
            for v in value:
                query_parameters.append(f"{key}={urllib.parse.quote(str(v))}")
        else:
            query_parameters.append(f"{key}={urllib.parse.quote(str(value))}")

    # Join all query parameters with "&" and append to the base_url
    url = base_url + "&".join(query_parameters)
    return url

# Example usage:
params = {
    "location": "changi city point",
    "term": "prata",
    "categories": "restaurants",
    "price": [2, 3],  # Multiple price levels
    "open_now": "true",
    "attributes": ["parking_lot", "liked_by_vegetarians"],  # Multiple attributes
    "limit": "20"
}

url = construct_yelp_url(params)
print(url)


https://api.yelp.com/v3/businesses/search?location=changi%20city%20point&term=prata&categories=restaurants&price=2&price=3&open_now=true&attributes=parking_lot&attributes=liked_by_vegetarians&limit=20
